# Parte 1: Implementación práctica

Marco Jurado 20308

In [225]:
#!pip install lightgbm
#!pip install --upgrade lightgbm
#!pip install category_encoders

In [226]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [227]:
data = pd.read_csv('fraud_feature_engineering_example.csv')
data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,trans_num,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net,first_time_at_merchant
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,4.97,4.97,0.00,0.0,True
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,107.23,107.23,0.00,0.0,True
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,220.11,220.11,0.00,0.0,True
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,45.00,45.00,0.00,0.0,True
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,41.96,41.96,0.00,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0,11619.63,72134.23,1014.44,11.0,False
1852390,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0,15224.47,87115.43,3942.78,25.0,False
1852391,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0,26233.12,165389.30,2978.91,29.0,False
1852392,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0,11787.71,90698.65,768.69,17.0,False


In [228]:
data.shape

(1852394, 27)

In [229]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 27 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   trans_date_trans_time         object 
 1   cc_num                        int64  
 2   merchant                      object 
 3   category                      object 
 4   amt                           float64
 5   first                         object 
 6   last                          object 
 7   gender                        object 
 8   street                        object 
 9   city                          object 
 10  state                         object 
 11  zip                           int64  
 12  lat                           float64
 13  long                          float64
 14  city_pop                      int64  
 15  job                           object 
 16  dob                           object 
 17  trans_num                     object 
 18  unix_time             

Eliminaremos algunas columnas irrelevantes

In [230]:
columns_to_drop = ['first', 'last', 'city', 'state', 'zip', 'city_pop', 'job', 'dob']
data.drop(columns_to_drop, axis=1, inplace=True)

In [231]:
data.head()

,trans_date_trans_time,cc_num,merchant,category,amt,gender,street,lat,long,trans_num,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net,first_time_at_merchant
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,F,561 Perry Cove,36.0788,-81.1781,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0,4.97,4.97,0.0,0.0,True
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,F,43039 Riley Greens Suite 393,48.8878,-118.2105,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0,107.23,107.23,0.0,0.0,True
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,M,594 White Dale Suite 530,42.1808,-112.2620,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0,220.11,220.11,0.0,0.0,True
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,M,9443 Cynthia Court Apt. 038,46.2306,-112.1138,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0,45.00,45.00,0.0,0.0,True
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,M,408 Bradley Rest,38.4207,-79.4629,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0,41.96,41.96,0.0,0.0,True


In [232]:
data.describe()

,cc_num,amt,lat,long,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net
count,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06,1.852394e+06
mean,4.173860e+17,7.006357e+01,3.853931e+01,-9.022783e+01,1.358674e+09,3.853898e+01,-9.022794e+01,5.210015e-03,4.153689e+03,4.530560e+04,3.762028e+02,4.567241e+00
std,1.309115e+18,1.592540e+02,5.071470e+00,1.374789e+01,1.819508e+07,5.105604e+00,1.375969e+01,7.199217e-02,3.909005e+03,3.586752e+04,7.253531e+02,4.575502e+00
min,6.041621e+10,1.000000e+00,2.002710e+01,-1.656723e+02,1.325376e+09,1.902742e+01,-1.666716e+02,0.000000e+00,1.000000e+00,1.020000e+00,0.000000e+00,0.000000e+00
25%,1.800429e+14,9.640000e+00,3.466890e+01,-9.679800e+01,1.343017e+09,3.474012e+01,-9.689944e+01,0.000000e+00,1.344790e+03,1.734142e+04,9.020000e+00,1.000000e+00
50%,3.521417e+15,4.745000e+01,3.935430e+01,-8.747690e+01,1.357089e+09,3.936890e+01,-8.744069e+01,0.000000e+00,3.071990e+03,3.743910e+04,7.589000e+01,3.000000e+00
75%,4.642255e+15,8.310000e+01,4.194040e+01,-8.015800e+01,1.374581e+09,4.195626e+01,-8.024511e+01,0.000000e+00,5.738470e+03,6.472088e+04,4.259800e+02,7.000000e+00
max,4.992346e+18,2.894890e+04,6.669330e+01,-6.795030e+01,1.388534e+09,6.751027e+01,-6.695090e+01,1.000000e+00,4.326189e+04,2.190868e+05,1.204718e+04,4.800000e+01


In [233]:
data["merchant"].value_counts()

fraud_Kilback LLC                        6262
fraud_Cormier LLC                        5246
fraud_Schumm PLC                         5195
fraud_Kuhn LLC                           5031
fraud_Boyer PLC                          4999
                                         ... 
fraud_Douglas, DuBuque and McKenzie      1101
fraud_Treutel-King                       1098
fraud_Satterfield-Lowe                   1095
fraud_Hahn, Douglas and Schowalter       1091
fraud_Ritchie, Bradtke and Stiedemann    1090
Name: merchant, Length: 693, dtype: int64

In [234]:
data["is_fraud"].value_counts()

0    1842743
1       9651
Name: is_fraud, dtype: int64

In [235]:
data["cc_num"].value_counts()

6538441737335434       4392
30270432095985         4392
6538891242532018       4386
4364010865167176       4386
4642255475285942       4386
                       ... 
4714017207228610634       7
4352307151555405069       7
180097223252063           7
6011732631674654          7
4295296907373             6
Name: cc_num, Length: 999, dtype: int64

In [236]:
data["trans_date_trans_time"] = pd.to_datetime(data["trans_date_trans_time"])

In [237]:
data = data.sort_values(by='trans_date_trans_time')

In [238]:
data["cc_num"] = data["cc_num"].astype(str)

In [239]:
data["trans_num"].value_counts(dropna=False)

0b242abb623afc578575680df30655b9    1
ad44105b2defec28687e0c0c00db0dd2    1
fff81009b323191d0a427a2af21e5bc7    1
2341854d7594722011878c08ba3819dd    1
f7f00d638b16a8b1045d8a8978b607bd    1
                                   ..
74745c7e44a5172e919b2ea2538d0edb    1
71d44820f944faa2ed912043e859df48    1
986d5219c856c1ddcafe1a4ad35b844e    1
cf4fca8a9847c75240202201324dd34f    1
1765bb45b3aa3224b4cdcb6e7a96cee3    1
Name: trans_num, Length: 1852394, dtype: int64

Vamos a convertir aquellas columnas que no sean datos numericos con un encoder

In [240]:
columns_to_encode = ['merchant', 'category', 'gender', 'street', 'trans_num', 'cc_num']

with open('label_encodings.txt', 'w') as file:
    for col in columns_to_encode:
        encoder = LabelEncoder()
        data[col + '_encoded'] = encoder.fit_transform(data[col])
        mappings = {index: label for index, label in enumerate(encoder.classes_)}
        file.write(f"{col} mappings:\n")
        for key, value in mappings.items():
            file.write(f"{key}: {value}\n")
        file.write("\n")

data.drop(columns=columns_to_encode, axis=1, inplace=True)

In [241]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1852394 entries, 0 to 1852393
Data columns (total 19 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   trans_date_trans_time         datetime64[ns]
 1   amt                           float64       
 2   lat                           float64       
 3   long                          float64       
 4   unix_time                     int64         
 5   merch_lat                     float64       
 6   merch_long                    float64       
 7   is_fraud                      int64         
 8   amt_month                     float64       
 9   amt_year                      float64       
 10  amt_month_shopping_net_spend  float64       
 11  count_month_shopping_net      float64       
 12  first_time_at_merchant        bool          
 13  merchant_encoded              int32         
 14  category_encoded              int32         
 15  gender_encoded                in

In [242]:
non_numeric_columns = data.select_dtypes(include=['object']).columns

print("Columnas no numéricas que podrían necesitar codificación:")
non_numeric_columns

Columnas no numéricas que podrían necesitar codificación:


Index([], dtype='object')

In [243]:
data

,trans_date_trans_time,amt,lat,long,unix_time,merch_lat,merch_long,is_fraud,amt_month,amt_year,amt_month_shopping_net_spend,count_month_shopping_net,first_time_at_merchant,merchant_encoded,category_encoded,gender_encoded,street_encoded,trans_num_encoded,cc_num_encoded
0,2019-01-01 00:00:18,4.97,36.0788,-81.1781,1325376018,36.011293,-82.048315,0,4.97,4.97,0.00,0.0,True,514,8,0,576,80326,119
1,2019-01-01 00:00:44,107.23,48.8878,-118.2105,1325376044,49.159047,-118.186462,0,107.23,107.23,0.00,0.0,True,241,4,0,439,227462,924
2,2019-01-01 00:00:51,220.11,42.1808,-112.2620,1325376051,43.150704,-112.154481,0,220.11,220.11,0.00,0.0,True,390,0,1,610,1169030,491
3,2019-01-01 00:01:16,45.00,46.2306,-112.1138,1325376076,47.034331,-112.561071,0,45.00,45.00,0.00,0.0,True,360,2,1,945,777909,285
4,2019-01-01 00:03:06,41.96,38.4207,-79.4629,1325376186,38.674999,-78.632459,0,41.96,41.96,0.00,0.0,True,297,9,1,422,1186866,451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,43.77,40.4931,-91.8912,1388534347,39.946837,-91.333331,0,11619.63,72134.23,1014.44,11.0,False,507,5,1,573,1122239,191
1852390,2020-12-31 23:59:09,111.84,29.0393,-95.4401,1388534349,29.661049,-96.186633,0,15224.47,87115.43,3942.78,25.0,False,264,7,1,583,235369,327
1852391,2020-12-31 23:59:15,86.88,46.1966,-118.9017,1388534355,46.658340,-119.715054,0,26233.12,165389.30,2978.91,29.0,False,496,7,0,138,783898,905
1852392,2020-12-31 23:59:24,7.99,44.6255,-116.4493,1388534364,44.470525,-117.080888,0,11787.71,90698.65,768.69,17.0,False,75,13,1,711,145946,512


Vamos a separar el dataset en 80%, 20%

In [244]:
data = pd.read_csv('fraud_feature_engineering_example.csv')
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

### LightGBM

In [245]:
print(X_train.isnull().sum())
print(X_train.describe())

trans_date_trans_time           0
cc_num                          0
merchant                        0
category                        0
amt                             0
first                           0
last                            0
gender                          0
street                          0
city                            0
state                           0
zip                             0
lat                             0
long                            0
city_pop                        0
job                             0
dob                             0
trans_num                       0
unix_time                       0
merch_lat                       0
merch_long                      0
amt_month                       0
amt_year                        0
amt_month_shopping_net_spend    0
count_month_shopping_net        0
first_time_at_merchant          0
dtype: int64
             cc_num           amt           zip           lat          long  \
count  1.111436e+06  1.1

In [246]:
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val)
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbose': -1
}
lgb_model = lgb.train(
    params,
    train_data,
    valid_sets=[valid_data],
    early_stopping_rounds=10,
    num_boost_round=100
)

TypeError: train() got an unexpected keyword argument 'early_stopping_rounds'

Entrenamos incrementalmente el modelo LightGBM

In [ ]:
lgb_model = lgb.train(params, valid_data, init_model=lgb_model, early_stopping_rounds=10, num_boost_round=100)

In [ ]:
y_pred = (lgb_model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')